## Extracción de datos música Punk

In [1]:
#Importar
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
#Acceso: client id, client secret
Client_ID = "bd39a5f17be2422090f8beab3ab83461"
Client_secret = "dacb9e8f9b1c4ad9be7dd105496bc81f"

"redirect URI = http://127.0.0.1:8000/callback"

'redirect URI = http://127.0.0.1:8000/callback'

In [3]:
# Autenticacion
auth_manager = SpotifyClientCredentials(client_id=Client_ID, client_secret=Client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [4]:
# Función para etraer todos los resultados
def extraer_datos_por_anios(sp, genres, anios, max_por_anio=1000):
    results = []
    artistas = set()  

    for anio in anios:
        print(f"\n Extrayendo datos del año {anio}")

        for genre in genres:
            print(f" Género: {genre}")

            offset = 0
            obtenidos = 0

            while obtenidos < max_por_anio:
                response = sp.search(
                    q=f'genre:{genre} year:{anio}',
                    type='track,album',
                    limit=50,
                    offset=offset
                )

                tracks = response['tracks']['items']
                albums = response['albums']['items']

                if not tracks and not albums:
                    break  

                # Canciones
                for item in tracks:
                    if obtenidos >= max_por_anio:
                        break

                    artista = item['artists'][0]['name']
                    artistas.add(artista)

                    results.append({
                        'name_track': item['name'],
                        'name_artist': artista,
                        'genre': genre,
                        'type': 'track',
                        'year': item['album']['release_date'][:4]
                    })
                    obtenidos += 1

                # Álbumes
                for item in albums:
                    if obtenidos >= max_por_anio:
                        break

                    artista = item['artists'][0]['name']
                    artistas.add(artista)

                    results.append({
                        'name_track': item['name'],
                        'name_artist': artista,
                        'genre': genre,
                        'type': 'album',
                        'year': item['release_date'][:4]
                    })
                    obtenidos += 1

                offset += 50  

            print(f" Total extraído {anio} ({genre}): {obtenidos}")

    return results, list(artistas)

In [7]:
genres = ['punk', 'ska punk']
anios = [2010, 2015, 2020, 2025]

results, artistas = extraer_datos_por_anios(
    sp=sp,
    genres=genres,
    anios=anios,
    max_por_anio=1000
)

print(len(results))
print(len(artistas))


 Extrayendo datos del año 2010
 Género: punk
 Total extraído 2010 (punk): 1000
 Género: ska punk
 Total extraído 2010 (ska punk): 2

 Extrayendo datos del año 2015
 Género: punk
 Total extraído 2015 (punk): 1000
 Género: ska punk
 Total extraído 2015 (ska punk): 6

 Extrayendo datos del año 2020
 Género: punk
 Total extraído 2020 (punk): 1000
 Género: ska punk
 Total extraído 2020 (ska punk): 9

 Extrayendo datos del año 2025
 Género: punk
 Total extraído 2025 (punk): 1000
 Género: ska punk
 Total extraído 2025 (ska punk): 0
4017
1424


In [6]:
# Extracción datos last.fm
api_key = "a03eaf07fb8aba070de6cbf312f88b25"
base_url = "http://ws.audioscrobbler.com/2.0/"

In [ ]:
# Función extraer datos
import requests
import time

def obtener_info_artistas_lastfm(artistas, api_key, delay=0.25):

    base_url = "http://ws.audioscrobbler.com/2.0/"
    resultados = []

    for artista in artistas:
        params = {
            "method": "artist.getInfo",
            "artist": artista,
            "api_key": api_key,
            "format": "json"
        }

        try:
            response = requests.get(base_url, params=params)
            data = response.json()

            if "artist" not in data:
                print(f"  {artista} No encontrado.")
                continue

            artist_data = data["artist"]

            resultados.append({
                "artista": artista,
                "biografia": artist_data.get("bio", {}).get("summary", ""),
                "Oyentes": int(artist_data.get("stats", {}).get("listeners", 0)),
                "N_Reproducciones": int(artist_data.get("stats", {}).get("playcount", 0)),
                "artistas_similares": [
                    a["name"] for a in artist_data.get("similar", {}).get("artist", [])
                ]
            })

            time.sleep(delay) 

        except Exception as e:
            print(f"Error con {artista}: {e}")

    return resultados

In [ ]:
info_artistas = obtener_info_artistas_lastfm(artistas, api_key)

print(len(info_artistas))
info_artistas[0]

In [ ]:
# Variables con datos obtenidos:
results
artistas
info_artistas